In [234]:
# 1、读取监督分类的遥感影像,可以认为是一个掩膜数据。
# 2、读取NDVI、NDGI、RVI植被指数指标影像，存储为中间变量。
# 3、根据样本的植被指数指标设定过滤指标，第一次筛选：K(NDGI[8-22])<0,生成掩膜数据一
# 4、根据样本的植被指数指标设定过滤指标，第二次筛选：K(NDGI[8-22至9-16])>0,生成掩膜数据二
# 5、根据样本的植被指数指标设定过滤指标，第三次筛选：RVI[08-02至09-16]在(0.07,0.17)以外,生成掩膜数据三
# 6、把三次掩膜数据进行叠加，有0的标记为0，其他的标记为1，生成新的掩膜数据总
# 7、掩膜数据总和之前最大似然法获取的作物分类影像行进运算，提升影像运算精度，导出最终分类影像。

# 1、读取监督分类的遥感影像,可以认为是一个掩膜数据。

In [1]:
from osgeo import gdal
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tif_class = r'E:\Desktop\Summary\种植结构提取-水稻\datas\MLSC\MLSC_2021-08-22_4074_S2DL.tif'
dataset = gdal.Open(tif_class)
bandclass = dataset.GetRasterBand(1)
dataclass = bandclass.ReadAsArray()
dataclass.astype('int16')  # 设置分类后的数据类型
width = dataset.RasterXSize
height = dataset.RasterYSize
projection = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
print('完成！！！')

完成！！！


In [3]:
classic = dataclass.reshape(-1)

# 2、读取NDVI、NDGI、RVI植被指数指标影像，存储为中间变量。

In [4]:
# 2、读取NDVI、NDGI、RVI植被指数指标影像，存储为中间变量。
#NDVI 读取栅格数据
dfndvi = pd.DataFrame()
ProFiles = os.listdir(r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDVI')
for item in tqdm(ProFiles):
    if item.lower().endswith('.tif'):
        src = (r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDVI\\{}'.format(item))
        dataset = gdal.Open(src)
        ndvi_data = dataset.GetRasterBand(1).ReadAsArray()
        ndvi_data = np.nan_to_num(ndvi_data, nan=999)
        dfndvi[item[9:14]] = (ndvi_data.reshape(-1))
dfndvit = dfndvi.T
dfndvit['x'] = range(6)
# dfndvit.plot(x='x', y=list(range(1, 12)))
# dfndvit.to_csv(r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDVI\ndvi.csv')

100%|██████████| 13/13 [00:00<00:00, 724.37it/s]


In [5]:
dfndgi = pd.DataFrame()
ProFiles = os.listdir(r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDGI')
for item in tqdm(ProFiles):
    if item[-4:] == '.tif':
        src = (r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDGI\\{}'.format(item))
        dataset = gdal.Open(src)
        ndgi_data = dataset.GetRasterBand(1).ReadAsArray()
        ndgi_data = np.nan_to_num(ndgi_data, nan=999)
        dfndgi[item[9:14]] = (ndgi_data.reshape(-1))
dfndgit = dfndgi.T
dfndgit['x'] = range(6)
# dfndgit.plot(x='x', y=list(range(1, 12)))
# dfndgit.to_csv(r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDGI\ndgi.csv')

100%|██████████| 7/7 [00:00<00:00, 350.89it/s]


In [6]:
dfrvi = pd.DataFrame()
ProFiles = os.listdir(r'E:\Desktop\Summary\种植结构提取-水稻\datas\RVI')
for item in tqdm(ProFiles):
    if item[-4:] == '.tif':
        src = (r'E:\Desktop\Summary\种植结构提取-水稻\datas\RVI\\{}'.format(item))
        dataset = gdal.Open(src)
        rvi_data = dataset.GetRasterBand(1).ReadAsArray()
        rvi_data = np.nan_to_num(rvi_data, nan=999)
        dfrvi[item[8:13]] = (rvi_data.reshape(-1))
dfrvit = dfrvi.T
dfrvit['x'] = range(6)
# dfrvit.plot(x='x', y=list(range(1, 12)))
# dfrvit.to_csv(r'E:\Desktop\Summary\种植结构提取-水稻\datas\RVI\rvi.csv')

100%|██████████| 13/13 [00:00<00:00, 620.57it/s]


In [7]:
dfrvit

,0,1,2,3,4,5,6,7,8,9,...,203826,203827,203828,203829,203830,203831,203832,203833,203834,x
07-03,999.0,0.708869,0.672230,0.666888,0.669577,0.678208,0.690652,0.690652,0.694242,0.708651,...,0.682944,0.689519,0.688557,0.660290,0.660290,0.570157,0.489753,0.420241,999.0,0
08-02,999.0,0.096945,0.162025,0.149761,0.167720,0.170813,0.189131,0.189131,0.135271,0.115567,...,0.170451,0.234244,0.233996,0.127119,0.127119,0.082522,0.071731,0.078804,999.0,1
08-22,999.0,0.041948,0.058155,0.046536,0.052191,0.058699,0.095940,0.095940,0.104308,0.061321,...,0.076092,0.099560,0.075305,0.046975,0.046975,0.045636,0.040021,0.045964,999.0,2
09-16,999.0,0.110092,0.078486,0.076406,0.075494,0.096018,0.153597,0.153597,0.133333,0.116861,...,0.108523,0.084171,0.071934,0.063236,0.063236,0.062044,0.061353,0.066432,999.0,3
10-06,999.0,0.521097,0.349306,0.311470,0.333537,0.350955,0.478384,0.478384,0.641994,0.660767,...,0.358219,0.148340,0.173434,0.303219,0.303219,0.434677,0.400683,0.230296,999.0,4
10-21,999.0,0.664099,0.632379,0.634213,0.621841,0.617352,0.597816,0.597816,0.638693,0.623410,...,0.339679,0.192552,0.231513,0.287465,0.287465,0.363985,0.342499,0.253822,999.0,5


# 3、根据样本的植被指数指标设定过滤指标，第一次筛选：K(NDGI[8-22])<0,生成掩膜数据一

In [8]:
# 获取NDGI的斜率
# 根据条件一筛选
# dfndgit.shape
x_data = dfndgit['x'].values
NDGI_data = []
for i in range(dfndgit.shape[1] - 1):
    tmp = dfndgit[i].values
    NDGI_data.append(tmp)

In [246]:
slope_ndgi = []
for item in NDGI_data:
    tmp1 = np.gradient(item, x_data)
    slope_ndgi.append(tmp1)

In [ ]:
dfndgi

In [253]:
s_ndgi = pd.DataFrame(slope_ndgi)
# mask1 = np.where((s_ndgi[2] >=0&s_ndgi[3] <=0)|(s_ndgi[3] >=0&s_ndgi[4] <=0), 1, 0)
mask1 = np.where(((s_ndgi[2] >=0)&(s_ndgi[3] <=0)|(s_ndgi[3] >=0)&(s_ndgi[4] <=0)), 1, 0)

In [254]:
mask1.sum()

155314

# 4、根据样本的植被指数指标设定过滤指标，第二次筛选：K(NDGI[10-02至9-16])>0,生成掩膜数据二

In [175]:
mask2 = np.where(((s_ndgi[3] > 0) & (s_ndgi[4] > 0)), 0, 1)
mask2

array([1, 1, 1, ..., 1, 1, 1])

# 5、根据样本的植被指数指标设定过滤指标，第三次筛选：RVI[08-02至09-16]在(0.07,0.17)以外,生成掩膜数据三

In [23]:
type(dfrvi)

pandas.core.frame.DataFrame

In [34]:
dfrvi1 = dfrvi.rename(columns={'7-03_': 0, '8-02_': 1, '8-22_': 2, '9-16_': 3, '0-06_': 4, '0-21_': 5})

In [ ]:
dfrvi1

In [ ]:
mask3 = np.where(((dfrvi1[1] >= 0.07) & (dfrvi1[1] <= 0.17) & (dfrvi1[2] >= 0.07) & (dfrvi1[2] < 0.17) & (
        dfrvi1[3] >= 0.07) & (dfrvi1[3] < 0.17)), 1, 0)
mask3

# 6、把三次掩膜数据进行叠加，有0的标记为0，其他的标记为1，生成新的掩膜数据总

In [135]:
result_array = mask1.copy()
result_array = np.where(mask1 == 0, 0, result_array)
result_array = np.where(mask2 == 0, 0, result_array)
result_array = np.where(mask3 == 0, 0, result_array)
result_array

array([0, 0, 0, ..., 0, 0, 0])

# 7、掩膜数据总和之前最大似然法获取的作物分类影像行进运算，提升影像运算精度，导出最终分类影像。

In [ ]:
# classic
# np.where(((classic == 0) and (mask1 == 1)), 0, 1)

In [ ]:
type(result_array)

In [214]:
classic.astype('int16')

array([1, 0, 0, ..., 0, 0, 1], dtype=int16)

In [255]:
mask = np.where(((classic == 0) & (mask1 == 1)), 0, 1)
mask.sum()

143927

In [216]:
result_array.sum()

4219

In [ ]:
band=mask.reshape((height,width))
plt.imshow(band)
plt.savefig(r'E:\Desktop\Summary\种植结构提取-水稻\datas\LineChart\result2.png',dpi=600)

In [256]:
tif_class = r'E:\Desktop\Summary\种植结构提取-水稻\datas\MLSC\MLSC_2021-08-22_4074_S2DL.tif'
dataset = gdal.Open(tif_class)
bandclass = dataset.GetRasterBand(1)
dataclass = bandclass.ReadAsArray()
dataclass.astype('int16')  # 设置分类后的数据类型
width = dataset.RasterXSize
height = dataset.RasterYSize
projection = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
print('完成！！！')

完成！！！


In [ ]:
classic.sum()

In [257]:
result_path=r'E:\Desktop\Summary\种植结构提取-水稻\datas\result4sampel1.tif'
target_ds = gdal.GetDriverByName('GTiff').Create(result_path, xsize=width, ysize=height, bands=1,
                                                 eType=gdal.GDT_Int16)
target_ds.SetGeoTransform(dataset.GetGeoTransform())
target_ds.SetProjection(dataset.GetProjection())
del dataset
target_ds.GetRasterBand(1).SetNoDataValue(9999)
target_ds.GetRasterBand(1).WriteArray(band)
target_ds.FlushCache()